In [5]:
#Slight Improvement from previous
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate, PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector 

chat = ChatOpenAI(
    #temperature is a hyperparameter that controls the randomness of the generated text. 
    temperature = 0.1,
    streaming = True,
    callbacks = [
        StreamingStdOutCallbackHandler(),
    ],
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

###
###
###Phase 1
###
###


example_prompt = PromptTemplate.from_template(template="Human: {question}\nAI: {answer}")


example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=80,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables = ["country"],
)

prompt.format(country = "Brazil")

'Human: What do you know about France?\nAI: \n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

In [7]:
###
###
###Phase 2
###
###

#need this bad boy package
from typing import Any, Dict, List
from langchain.prompts.example_selector.base import BaseExampleSelector

'''

Getting the BaseExampleSelector as a parameter means
RandomExampleSelector will take all the methods and attributes
of BaseExampleSelector

'''

class RandomExampleSelector(BaseExampleSelector):
    #self means RandomExampleSelector
    #constructor method Maybe used in 
    def __init__(self,examples):
        self.examples = examples

    def select_examples(self, input_variables):
        from random import choice
        #return only one from the list
        #choice(examples) selects only one element from examples list
        return [choice(examples)]
    
    #should not return anything
    def add_example(self, example):
        self.examples.append(example)
    

#do not need the example_template and max_length attribute
example_selector = RandomExampleSelector(
    examples=examples
)
    

prompt.format(country = "Brazil")
    

'Human: What do you know about France?\nAI: \n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'